# Operaciones CRUD en Neo4j

In [1]:
%load_ext icypher
%matplotlib inline

In [2]:
!curl -s http://localhost:7474 | grep neo4j_version

  "neo4j_version" : "4.2.4",


## Borrado de todos los nodos y relaciones de la base de datos

In [3]:
%%cypher
MATCH (n)
OPTIONAL MATCH (n)-[r]-()
DELETE n,r

[]

## Crear 1 nodo

In [4]:
%%cypher
CREATE (ann:Person { name: 'Ann' }) 
RETURN ann

[{'ann': (_13:Person {name: 'Ann'})}]

In [5]:
%cypher CREATE (dan:Person { name: 'Dan' }) RETURN dan

[{'dan': (_14:Person {name: 'Dan'})}]

## Crear relaciones

In [6]:
%%cypher
MATCH (a:Person { name: 'Ann' }), 
    (b:Person { name: 'Dan' })
CREATE (a)-[:KNOWS]->(b)

[]

## Crear un nodo con MERGE

Sólo se crea si no existe previamente

In [7]:
%%cypher
MERGE (n:Person { name: 'Victor' , age : 20}) RETURN n

[{'n': (_15:Person {age: 20, name: 'Victor'})}]

Sólo se crea si no existe previamente## Crear una relación con MERGE

Sólo se crea si no existe previamente

In [8]:
%%cypher
MATCH (a:Person { name: 'Ann' })
MATCH (b:Person { name: 'Dan' })
MERGE (a)-[r:KNOWS]->(b)
RETURN a,type(r),b

[{'a': (_13:Person {name: 'Ann'}),
  'type(r)': 'KNOWS',
  'b': (_14:Person {name: 'Dan'})}]

In [9]:
%%cypher
MATCH (a:Person { name: 'Ann' }), (b:Person { name: 'Victor' })
MERGE (a)-[r:KNOWS]->(b)
RETURN a,type(r),b

[{'a': (_13:Person {name: 'Ann'}),
  'type(r)': 'KNOWS',
  'b': (_15:Person {age: 20, name: 'Victor'})}]

## Nodos y relaciones al mismo tiempo

In [10]:
%%cypher
CREATE (Felix:Person {name:'Felix'})
CREATE (Mariam:Person {name:'Mariam',
                       born:1964})
CREATE
  (Felix)-[:KNOWS {since:2010}]->(Mariam)
;

[]

## Encontrar patrones

In [11]:
%%cypher 
MATCH (n:Person) 
RETURN n.name

[{'n.name': 'Ann'},
 {'n.name': 'Dan'},
 {'n.name': 'Victor'},
 {'n.name': 'Felix'},
 {'n.name': 'Mariam'}]

In [12]:
%%cypher
MATCH (n:Person { name: 'Ann' }) 
RETURN n.name

[{'n.name': 'Ann'}]

In [13]:
%%cypher
MATCH (n:Person) 
WHERE n.name = 'Ann'
RETURN n.name

[{'n.name': 'Ann'}]

In [14]:
%%cypher 
MATCH (n:Person) 
WHERE n.age > 10
RETURN n

[{'n': (_15:Person {age: 20, name: 'Victor'})}]

In [15]:
%%cypher 
MATCH (n:Person { name: 'Ann' })-[r:KNOWS]-(neighbors) 
RETURN n,neighbors

[{'n': (_13:Person {name: 'Ann'}),
  'neighbors': (_15:Person {age: 20, name: 'Victor'})},
 {'n': (_13:Person {name: 'Ann'}), 'neighbors': (_14:Person {name: 'Dan'})}]

In [16]:
%%cypher
MATCH (h:Person)<-[:KNOWS]-(:Person)-[:KNOWS]->(Person) 
RETURN h.name AS Name, COUNT(*) AS count
ORDER BY count DESC
LIMIT 10

[{'Name': 'Dan', 'count': 1}, {'Name': 'Victor', 'count': 1}]

In [17]:
%%cypher
MATCH (n:Person { name: 'Victor' })-[:KNOWS*1..3]-(neighbors) 
RETURN DISTINCT n, neighbors

[{'n': (_15:Person {age: 20, name: 'Victor'}),
  'neighbors': (_14:Person {name: 'Dan'})},
 {'n': (_15:Person {age: 20, name: 'Victor'}),
  'neighbors': (_13:Person {name: 'Ann'})}]

In [18]:
%%cypher
MATCH (n:Person { name: 'Dan' })-[:KNOWS*1..3]-(neighbors) 
RETURN n, collect(DISTINCT neighbors)

[{'n': (_14:Person {name: 'Dan'}),
  'collect(DISTINCT neighbors)': [(_15:Person {age: 20, name: 'Victor'}),
   (_13:Person {name: 'Ann'})]}]

## Actualizar 1 nodo

In [19]:
%%cypher
MATCH (n:Person {name : "Ann"})
SET n.hair = "Brown"

[]

## Borrar 1 nodo

In [20]:
%%cypher
CREATE (n:Person {name : "Alex"})
RETURN n;

[{'n': (_18:Person {name: 'Alex'})}]

In [21]:
%%cypher
MATCH (Alex:Person {name:"Alex"})
DELETE Alex

[]

## Algoritmos

In [22]:
%%cypher
MATCH p=shortestPath(
  (a:Person { name: 'Ann' })-[:KNOWS]-(b:Person { name: 'Dan'})
)
RETURN p

[{'p': (Ann)-[:KNOWS {}]->(Dan)}]

In [23]:
%%cypher
MATCH p=shortestPath(
  (a:Person { name: 'Victor' })-[:KNOWS*1..3]-(b:Person { name: 'Dan'})
)
RETURN p

[{'p': (Victor)<-[:KNOWS {}]-(Ann)-[:KNOWS {}]->(Dan)}]